In [1]:
import preprocessing as pp
import deter_mod as dm
import pandas as pd
import numpy as np
import time 
import xpress as xp # FICO Xprerss Solver
xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')

# Main

In [ ]:
# Read and Preprocess the data
customer_df, candidate_df, supplier_df, vehicle_df, distance_w_to_s_df, distance_w_to_c_df, demand_cus_period_df, demand_cus_period_scene_df = pp.read_and_prep_data()


time_limit_s = 3600
time_obj_gap_dict = {}

############ Change you clusters list here ############
n_clusters_list = [300]


for each_n_clusters in n_clusters_list:
    print(f'Now running Case N_cluster = {each_n_clusters}')
    # Customer Clustering
    customer_df, cluster_center_df = pp.const_cluster_by_cus_loc(customer_df, n_clusters=each_n_clusters, size_min=np.floor(len(customer_df)/each_n_clusters), size_max=400, random_state=42)

    agg_dem_cus_period_df = pp.agg_dem_cus_period(demand_cus_period_df,customer_df)

    agg_dem_cus_period_scene_df = pp.agg_dem_cus_period_scene(demand_cus_period_scene_df, customer_df)

    distance_w_to_cluster_df = pp.create_dis_mat_df(candidate_df, cluster_center_df,'cityblock')

    # Create Cost
    cost_w_to_cluster = pp.calculate_cost_from_w_to_cluster(distance_w_to_cluster_df, vehicle_df)
    cost_w_to_s = pp.calculate_cost_from_w_to_s(distance_w_to_s_df, vehicle_df, supplier_df)

    solve_time, obj_value, mip_gap_percent, x_matrix, y_matrix, v_matrix, z_matrix = dm.deter_model(agg_dem_cus_period_df, 
                                                                                                    candidate_df, 
                                                                                                    supplier_df, 
                                                                                                    cluster_center_df,
                                                                                                    cost_w_to_s,
                                                                                                    cost_w_to_cluster,
                                                                                                    time_limit_s)

    time_obj_gap_dict[each_n_clusters] = [solve_time, obj_value, mip_gap_percent]

Now running Case N_cluster = 300
FICO Xpress v9.3.5, Hyper, solve started 4:51:32, Feb 13, 2025
Heap usage: 3748MB (peak 3748MB, 1232MB system)
Minimizing MILP MEWLP_Deterministic using up to 11 threads and up to 18GB memory, with these control settings:
MAXTIME = 3600
OUTPUTLOG = 1
MIPRELSTOP = .001
Original problem has:
   6633890 rows      6838040 cols     25549920 elements   1324840 entities
Presolved problem has:
   1351546 rows      1557187 cols     11016351 elements   1323987 entities
LP relaxation tightened
Presolve finished in 71 seconds
Heap usage: 5446MB (peak 9955MB, 1232MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  8.00e+06] / [ 7.32e-04,  1.05e+03]
  RHS and bounds [min,max] : [ 1.00e+00,  1.53e+07] / [ 1.00e+00,  1.50e+04]
  Objective      [min,max] : [ 2.45e-04,  6.25e+06] / [ 3.13e-02,  6.88e+06]
Autoscaling applied Curtis-Reid scaling

Will try to keep branch and bound tree memory usa

In [ ]:
solve_time

553.8658890724182

In [ ]:
obj_value

39429889.638179846

In [ ]:
mip_gap_percent

0.09325707161602334

In [ ]:
asefasdf

NameError: name 'asefasdf' is not defined

In [8]:
import plotly.graph_objects as go

In [10]:
op_dict = {50: [198.49877619743347, 40013195.792221814, 0.09304184680571338],

 60: [168.20213294029236, 39769222.25899721, 0.09811661715458345],

 70: [481.5577218532562, 39956012.0455292, 9.903609429801749e-05],

 80: [368.98180413246155, 40128740.45679453, 0.09021935342220361],

 90: [200.23449110984802, 39269838.29936974, 0.09249473846241124],
 
 100: [309.9046268463135, 39036803.40854998, 0.08832317372508923],
 
 150: [553.8658890724182, 39429889.638179846, 0.09325707161602334],

 200: [921.9919128417969, 39170179.753491834, 0.09610256188168916],
 
 250: [1159.5304970741272, 39442627.54822727, 0.06706048501163756]}

op_x = list(op_dict.keys())
op_rt_y = [op_dict[x][0] for x in op_x]
op_obj_y = [op_dict[x][1] for x in op_x]

In [16]:
fig_obj = go.Figure()
fig_rt = go.Figure()
fig_rt.add_trace(go.Scatter(x = op_x, y = op_rt_y))
fig_obj.add_trace(go.Scatter(x = op_x, y = op_obj_y))

fig_rt.update_layout(title = 'Running Time when adjusting Numbers of Clusters', xaxis_title = 'N. of Clusters', yaxis_title = 'Running Time (s)')
fig_obj.update_layout(title = 'Objective Value when adjusting Numbers of Clusters', xaxis_title = 'N. of Clusters', yaxis_title = 'Objective Value (£)')
fig_rt.show()
fig_obj.show()

In [ ]:
time_obj_gap_dict

{250: [1159.5304970741272, 39442627.54822727, 0.06706048501163756]}

In [ ]:
asdfsdf

NameError: name 'asdfsdf' is not defined

{50: [198.49877619743347, 40013195.792221814, 0.09304184680571338],

 60: [168.20213294029236, 39769222.25899721, 0.09811661715458345],

 70: [481.5577218532562, 39956012.0455292, 9.903609429801749e-05],

 80: [368.98180413246155, 40128740.45679453, 0.09021935342220361],

 90: [200.23449110984802, 39269838.29936974, 0.09249473846241124],
 
 100: [309.9046268463135, 39036803.40854998, 0.08832317372508923]
 
 150: [553.8658890724182, 39429889.638179846, 0.09325707161602334]

 200: [921.9919128417969, 39170179.753491834, 0.09610256188168916]
 
 250: [1159.5304970741272, 39442627.54822727, 0.06706048501163756]}

# Appendix

In [ ]:
customer_df, candidate_df, supplier_df, vehicle_df, distance_w_to_s_df, distance_w_to_c_df, demand_cus_period_df, demand_cus_period_scene_df = pp.read_and_prep_data()

# 2 methods for clustering

# customer_df, cluster_center_df = pp.ori_cluster_by_cus_loc(customer_df, n_clusters=50, random_state=42, n_init=10)
customer_df, cluster_center_df = pp.const_cluster_by_cus_loc(customer_df, n_clusters=50, size_min=4, size_max=400, random_state=42)

agg_dem_cus_period_df = pp.agg_dem_cus_period(demand_cus_period_df,customer_df)

agg_dem_cus_period_scene_df = pp.agg_dem_cus_period_scene(demand_cus_period_scene_df, customer_df)

distance_w_to_cluster_df = pp.create_dis_mat_df(candidate_df, cluster_center_df,'cityblock')

# Create Cost
cost_w_to_cluster = pp.calculate_cost_from_w_to_cluster(distance_w_to_cluster_df, vehicle_df)
cost_w_to_s = pp.calculate_cost_from_w_to_s(distance_w_to_s_df, vehicle_df, supplier_df)

In [ ]:
solve_time, obj_value, mip_gap_percent, x_matrix, y_matrix, v_matrix, z_matrix = dm.deter_model(agg_dem_cus_period_df, 
                                                                                                candidate_df, 
                                                                                                supplier_df, 
                                                                                                cluster_center_df,
                                                                                                cost_w_to_s,
                                                                                                cost_w_to_cluster,
                                                                                                time_limit_s = 3600)

In [ ]:
# agg_dem_cus_period_df

0        1        2        3         4         5  \
Cluster ProductIndex                                                           
0       0              2021.0   3465.0   5257.0   7080.0    9786.0   11550.0   
        1              1534.0   2800.0   4719.0   5714.0    7265.0    8650.0   
        2             11000.0  23163.0  34163.0  44535.0   55060.0   63622.0   
        3              1634.0   3036.0   4858.0   6624.0    8382.0    9685.0   
1       0             20486.0  41680.0  60705.0  81991.0  103064.0  125688.0   
...                       ...      ...      ...      ...       ...       ...   
48      3              4413.0   8950.0  13452.0  17956.0   21926.0   28344.0   
49      0              3862.0   7559.0  13157.0  17046.0   19668.0   23445.0   
        1              3093.0   6015.0   9957.0  13420.0   15093.0   18533.0   
        2             16549.0  31675.0  52377.0  70669.0   82038.0   97013.0   
        3              4372.0   9619.0  13894.0  18882.0   22529.0   26963.0   

                             6         7         8         9  
Cluster ProductIndex                                          
0       0              13392.0   15250.0   17012.0   18793.0  
        1              10662.0   11829.0   13533.0   14765.0  
        2              76527.0   90310.0   99354.0  110543.0  
        3              10957.0   12378.0   14355.0   15838.0  
1       0             143016.0  164391.0  184797.0  206049.0  
...                        ...       ...       ...       ...  
48      3              30611.0   37286.0   40001.0   44824.0  
49      0              28508.0   32636.0   36398.0   39976.0  
        1              21409.0   26742.0   29609.0   32567.0  
        2             120789.0  129908.0  147702.0  166654.0  
        3              29497.0   35264.0   39689.0   43450.0  

[200 rows x 10 columns]

In [ ]:
# agg_dem_cus_period_scene_df.reset_index().T

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
Cluster,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49.0,49.0,49.0,49.0,49.0,49.0,49.0,49.0,49.0,49.0
ProductIndex,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
PeriodIndex,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
0,2235.0,4192.0,6756.0,8244.0,13389.0,17362.0,19896.0,26075.0,28626.0,35301.0,...,4130.0,12441.0,16451.0,25300.0,29612.0,36848.0,42804.0,58561.0,73877.0,87412.0
1,1923.0,3335.0,4556.0,6122.0,8219.0,10655.0,13249.0,14449.0,16114.0,18114.0,...,4081.0,10312.0,14108.0,18114.0,20766.0,24209.0,28798.0,28301.0,40281.0,45224.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1866.0,2634.0,3732.0,4341.0,5163.0,4823.0,4572.0,3994.0,2606.0,1091.0,...,3757.0,7682.0,11643.0,10966.0,10953.0,12366.0,10308.0,8010.0,6777.0,2965.0
96,2110.0,3929.0,6729.0,8537.0,14033.0,18164.0,19825.0,26361.0,28302.0,34507.0,...,5144.0,10047.0,18543.0,27201.0,27322.0,42781.0,46649.0,56200.0,65458.0,67232.0
97,2088.0,3560.0,5218.0,7213.0,10653.0,11988.0,13620.0,16295.0,17221.0,19408.0,...,4521.0,9459.0,16137.0,19125.0,20454.0,30752.0,29767.0,42928.0,36597.0,52438.0
98,1956.0,3422.0,4653.0,7030.0,7805.0,10290.0,10457.0,11886.0,12726.0,13447.0,...,4520.0,8265.0,12259.0,15785.0,21053.0,24747.0,23404.0,27148.0,26114.0,29176.0


In [ ]:
# agg_dem_cus_period_df.max()

0     499586.0
1     991057.0
2    1502986.0
3    2016177.0
4    2553411.0
5    3001479.0
6    3548115.0
7    4049779.0
8    4531440.0
9    5027653.0
dtype: float64

In [ ]:
# def create_dis_mat_df(depart_df:pd.DataFrame, arrive_df:pd.DataFrame, method: 'cityblock'):
#     # Assuming warehouse_df and customer_df have columns ['x', 'y']
#     depart_coords = depart_df[['CandidateEasting', 'CandidateNorthing']].values
#     arrive_coords = arrive_df[['CustomerEasting', 'CustomerNorthing']].values

#     # Compute Euclidean distance matrix
#     distance_matrix = cdist(depart_coords, arrive_coords, metric=method)/1000
#     # display(distance_matrix)
#     # Convert to a Pandas DataFrame for better readability
#     distance_df = pd.DataFrame(distance_matrix, 
#                             index=depart_df.index, 
#                             columns=arrive_df.index)

#     return distance_df

# def calculate_cost_from_w_to_cluster(distance_df:pd.DataFrame, vehicle_df:pd.DataFrame):
#     cost_w_to_cluster = distance_df.copy()
#     for w in range(len(distance_df.index)):
#         for c in range(len(distance_df.columns)):
#             cost_w_to_cluster.at[w,c] = 2 * distance_df.at[w,c] * vehicle_df.at[2,'VehicleCostPerMileAndTonneOverall'] / 1000   
#     return cost_w_to_cluster

# def calculate_cost_from_w_to_s(distance_df:pd.DataFrame, vehicle_df:pd.DataFrame, supplier_df:pd.DataFrame):
#     cost_w_to_s = distance_df.copy()
#     for w in range(len(distance_df.index)):
#         for s in range(len(distance_df.columns)):
#             if supplier_df.at[s,'SupplierVehicleType'] == 0:
#                 cost_w_to_s.at[w,s] = 2 * distance_df.at[w,s] * vehicle_df.at[0,'VehicleCostPerMileAndTonneOverall'] / 1000   
#             elif supplier_df.at[s,'SupplierVehicleType'] == 1:
#                 cost_w_to_s.at[w,s] = 2 * distance_df.at[w,s] * vehicle_df.at[1,'VehicleCostPerMileAndTonneOverall'] / 1000   
#             else:
#                 pass
#     return cost_w_to_s

In [ ]:
# # Create the model
# model = xp.problem(name = 'MEWLP')
# # Sets and Notation
# S = list(supplier_df.index) # Supplier Index
# S_P0 = list(supplier_df[supplier_df['SupplierProductGroup'] == 0]['SupplierProductGroup'].index)
# S_P1 = list(supplier_df[supplier_df['SupplierProductGroup'] == 1]['SupplierProductGroup'].index)
# S_P2 = list(supplier_df[supplier_df['SupplierProductGroup'] == 2]['SupplierProductGroup'].index)
# S_P3 = list(supplier_df[supplier_df['SupplierProductGroup'] == 3]['SupplierProductGroup'].index)
# S_P_dict = {0:S_P0, 1:S_P1, 2:S_P2, 3:S_P3}

# W = list(candidate_df.index) # Warehouse Index
# C = list(cluster_center_df.index) # Cluster Index
# P = list(agg_dem_cus_period_df.reset_index()['ProductIndex'].unique())
# T = list(list(agg_dem_cus_period_df.columns))

# # Output Variables
# x = np.array([xp.var(f'x_{w}_{c}_{p}_{t}', vartype = xp.binary) for w in W for c in C for p in P for t in T], dtype = xp.npvar).reshape(len(W), len(C), len(P), len(T))
# y = np.array([xp.var(f'y_{w}_{t}', vartype = xp.binary) for w in W for t in T], dtype = xp.npvar).reshape(len(W), len(T))
# o = np.array([xp.var(f'o_{w}', vartype = xp.binary) for w in W], dtype = xp.npvar).reshape(len(W))
# v = np.array([xp.var(f'v_{w}_{c}_{p}_{t}', vartype = xp.continuous, lb = 0) for w in W for c in C for p in P for t in T], dtype = xp.npvar).reshape(len(W), len(C), len(P), len(T))
# z = np.array([xp.var(f'z_{w}_{s}_{t}', vartype = xp.continuous, lb = 0) for w in W for s in S for t in T], dtype = xp.npvar).reshape(len(W), len(S), len(T))

# model.addVariable(x, y, o, v, z)

# # Constraints 
# for w in W:
#     Capacity_W = candidate_df.loc[w,'Capacity']
#     for t in T:
#         model.addConstraint(xp.constraint(o[w] >= y[w,t]))
#         model.addConstraint(xp.constraint(xp.Sum(v[w,c,p,t] for c in C for p in P) <= Capacity_W * y[w,t]))
#         if t != 0:
#             model.addConstraint(xp.constraint(y[w, t] >= y[w,t-1]))    

#         for p in P:
#             model.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for s in S_P_dict[p]) == xp.Sum(v[w,c,p,t] for c in C)))
#             for c in C:
#                 model.addConstraint(xp.constraint(x[w,c,p,t] <= y[w,t]))  

# for t in T:
#     for c in C:
#         for p in P:
#             Demand = agg_dem_cus_period_df.loc[(c,p),t]
#             model.addConstraint(xp.constraint(xp.Sum(x[w,c,p,t] for w in W) == 1))

#             for w in W:
#                 model.addConstraint(xp.constraint(v[w,c,p,t] == Demand * x[w,c,p,t]))    

# for s in S:
#     Capacity_S = supplier_df.loc[s,'SupplierCapacity']
#     # Product_S = supplier_df.loc[s,'SupplierProductGroup']
#     for t in T:
#         model.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for w in W) <= Capacity_S))



# Setup_cost = xp.Sum(candidate_df.loc[w,'Setup'] * o[w] for w in W)
# Operating_cost = xp.Sum(candidate_df.loc[w,'Operating'] * y[w,t] for w in W for t in T)
# Tra_w_s_cost = xp.Sum(cost_w_to_s.loc[w,s] * z[w,s,t] for w in W for s in S for t in T)
# Tra_w_c_cost = xp.Sum(cost_w_to_cluster.loc[w,c] * v[w,c,p,t] for w in W for c in C for p in P for t in T)

# obj = Setup_cost + Operating_cost + Tra_w_s_cost + Tra_w_c_cost

# model.setObjective(obj, sense = xp.minimize)
# model.setControl('miprelstop', 1e-3)
# model.setControl('maxtime', 600)
# tic_time = time.time()
# # Solve the problem
# model.solve()
# toc_time = time.time()
# solve_time = toc_time - tic_time
# obj_value = model.getObjVal()

In [ ]:
asdfasd

NameError: name 'asdfasd' is not defined

In [ ]:
# candidate_df

,CandidateId,CandidateEasting,CandidateNorthing,Capacity,Setup,Operating
0,"""AB10""",392949.0,805109.0,4320000.0,5424000.0,542400.0
1,"""AB11""",394404.0,805438.0,4360000.0,5312000.0,531200.0
2,"""AB12""",393407.0,800946.0,4720000.0,4388000.0,438800.0
3,"""AB13""",385705.0,801808.0,4760000.0,4318000.0,431800.0
4,"""AB14""",383616.0,801140.0,5040000.0,3716000.0,371600.0
...,...,...,...,...,...,...
435,"""TD11""",379765.0,654777.0,6160000.0,1854000.0,185400.0
436,"""TD12""",384850.0,640188.0,6320000.0,1658000.0,165800.0
437,"""TD13""",377046.0,670810.0,6480000.0,1448000.0,144800.0
438,"""TD14""",393270.0,663526.0,7000000.0,1000000.0,100000.0


In [ ]:
# x_matrix = model.getSolution(x)
# y_matrix = model.getSolution(y)
# # v_matrix = model.getSolution(v)
# z_matrix = model.getSolution(z)

?557 Error: Integer solution is not available


SolverError: ?557 Error: Integer solution is not available

In [ ]:
# w_open_output = pd.DataFrame(y_matrix).astype(int)

In [ ]:
# Capacity_S = supplier_df.loc[0,'SupplierCapacity']

In [ ]:
# s_flow_output = pd.DataFrame()
# for s in S:
#     Capacity_S = supplier_df.loc[s,'SupplierCapacity']
#     if np.any((Capacity_S - pd.DataFrame(z_matrix[:,s,:,:].sum(axis = 0)).sum(axis = 0)) < 0) :
#         raise ValueError(f'Supplier {s} exceed its capacity.')
#     else:
#         for w in W:
#             sub_df = pd.DataFrame(z_matrix[w,s,:,:])
#             sub_df.reset_index(inplace= True)
#             sub_df.rename(columns = {'index':'ProductIndex'},inplace = True)
#             sub_df['CandidateIndex'] = w
#             sub_df['SupplierIndex'] = s

#             s_flow_output = pd.concat([s_flow_output, sub_df])            

# s_flow_output = s_flow_output.groupby(['SupplierIndex','CandidateIndex','ProductIndex']).sum()
    

IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed

In [ ]:
# Capacity_S - pd.DataFrame(z_matrix[:,s,:,:].sum(axis = 0)).sum(axis = 0)

0    -2490720.0
1    -6305945.0
2   -10312794.0
3   -14211844.0
4   -18125606.0
5   -22763834.0
6   -26216523.0
7   -31152999.0
8   -35121815.0
9   -38990205.0
dtype: float64

In [ ]:
# w_flow_output = pd.DataFrame()
# for w in W:
#     Capacity_W = candidate_df.loc[w,'Capacity']
#     if np.any((Capacity_W - pd.DataFrame(v_matrix[w,:,:,:].sum(axis = 0)).sum(axis = 0)) < 0) :
#         raise ValueError(f'Warehose {w} exceed its capacity.')
#     else:
#         for c in C:
#             sub_df = pd.DataFrame(v_matrix[w,c,:,:])
#             sub_df.reset_index(inplace= True)
#             sub_df.rename(columns = {'index':'ProductIndex'},inplace = True)
#             sub_df['CandidateIndex'] = w
#             sub_df['ClusterIndex'] = c

#             w_flow_output = pd.concat([w_flow_output, sub_df])

# w_flop_output = w_flow_output.groupby(['CandidateIndex','ClusterIndex','ProductIndex']).sum()

In [ ]:
# Capacity_W - w_flop_output.loc[(0),:].sum(axis = 0) == Capacity_W - pd.DataFrame(v_matrix[0,:,:,:].sum(axis = 0)).sum(axis = 0) 

0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
9    True
dtype: bool

In [ ]:
# np.any((Capacity_W - pd.DataFrame(v_matrix[0,:,:,:].sum(axis = 0)).sum(axis = 0)) < 0)

False

In [ ]:
# Capacity_W = candidate_df.loc[0,'Capacity']

In [ ]:
# Capacity_W

4320000.0

In [ ]:
# v_matrix[0,:,:,:].sum(axis = 1)

array([[1.02000e+02, 3.27900e+03, 4.89500e+03, 6.60000e+03, 4.89000e+02,
        1.08170e+04, 1.11040e+04, 1.32290e+04, 1.55370e+04, 1.73460e+04],
       [0.00000e+00, 7.44600e+03, 9.66700e+03, 1.37310e+04, 1.68120e+04,
        2.10110e+04, 9.92800e+03, 1.62250e+04, 1.22460e+04, 3.31950e+04],
       [3.26600e+03, 5.16340e+04, 8.21390e+04, 1.12395e+05, 3.17520e+04,
        5.43620e+04, 4.27950e+04, 2.01200e+04, 5.74890e+04, 9.26660e+04],
       [0.00000e+00, 7.67000e+02, 1.04500e+03, 1.38900e+03, 1.95000e+03,
        2.13400e+03, 7.59000e+02, 2.67700e+03, 3.23400e+03, 3.62700e+03],
       [0.00000e+00, 2.78380e+04, 4.18780e+04, 5.92310e+04, 7.02880e+04,
        1.22333e+05, 1.01312e+05, 1.13259e+05, 7.78110e+04, 5.96520e+04],
       [0.00000e+00, 8.43100e+03, 1.32440e+04, 1.75790e+04, 2.14810e+04,
        2.71530e+04, 3.05040e+04, 3.50890e+04, 3.94650e+04, 4.45620e+04],
       [0.00000e+00, 5.42910e+04, 8.32160e+04, 1.13934e+05, 1.38490e+05,
        0.00000e+00, 8.78480e+04, 0.00000e+00

In [ ]:
# pd.DataFrame(v_matrix[0,:,:,:].sum(axis = 1))

,0,1,2,3,4,5,6,7,8,9
0,102.0,3279.0,4895.0,6600.0,489.0,10817.0,11104.0,13229.0,15537.0,17346.0
1,0.0,7446.0,9667.0,13731.0,16812.0,21011.0,9928.0,16225.0,12246.0,33195.0
2,3266.0,51634.0,82139.0,112395.0,31752.0,54362.0,42795.0,20120.0,57489.0,92666.0
3,0.0,767.0,1045.0,1389.0,1950.0,2134.0,759.0,2677.0,3234.0,3627.0
4,0.0,27838.0,41878.0,59231.0,70288.0,122333.0,101312.0,113259.0,77811.0,59652.0
...,...,...,...,...,...,...,...,...,...,...
95,0.0,6474.0,8047.0,11628.0,15537.0,17665.0,15276.0,19632.0,25804.0,28390.0
96,0.0,23352.0,37579.0,47111.0,59507.0,73165.0,75408.0,97790.0,23799.0,26326.0
97,0.0,113043.0,34074.0,44298.0,51293.0,67159.0,75523.0,87678.0,34520.0,79521.0
98,0.0,39484.0,64441.0,88555.0,110529.0,580569.0,62064.0,67733.0,0.0,72093.0


0    4316632.0
1      84478.0
2      29270.0
3      40004.0
4     203419.0
5      32728.0
6      60999.0
7     332071.0
8    1161248.0
9     176376.0
dtype: float64

In [ ]:
# cover_output = agg_dem_cus_period_df.copy()
# cover_output.loc[:,:] = -1
# for c in C:
#     for p in P:
#         for t in T:
#             cover_from_w = np.where(x_matrix[:,c,p,t] != 0)[0]
#             cover_from_w_check = np.where(v_matrix[:,c,p,t] != 0)[0]
#             if np.size(cover_from_w) == 1 and cover_from_w == cover_from_w_check: # Check that only one w will c for p during t 
#                 cover_output.at[(c,p),t] = int(cover_from_w[0])
#             else:
#                 raise ValueError(f"More Than 2 warehouse covers customer {c} for product {p} during time {t}")

# cover_output = cover_output.astype('int')  

# if np.any(cover_output == -1):
#     raise ValueError('Some entry of this df has not filled in yet.')

In [ ]:
# cover_output.head(10)

0   1    2    3    4   5    6   7    8    9
Cluster ProductIndex                                               
0       0             437  92  163  349  261  87  191  77  255  213
        1             320   0    0    0    2   0    2   2    0    0
        2             388   0    0    0    2   0    0   0    0    0
        3               0   0    0    0    0   0    0   0    0    0
1       0             277   0    0    0    0   0    0   2    0    0
        1             263   0    0    0    0   0    0   2    0    0
        2             150   0    0    0    0   0    2   0  385    0
        3              10  10   10   10   10  10   10  10   10   10
2       0              94   0    0    0    0   0    0   2    0    0
        1              28   0    0    0    0   0    0   0    0    0

In [ ]:
# np.size(np.where(x_matrix[:,0,1,2] != 0)[0])

1

In [ ]:
# np.where(v_matrix[:,0,1,2] != 0)[0]

array([0])

In [ ]:
# v_matrix[:,0,1,2]

array([288.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   

In [ ]:
# agg_dem_cus_period_df

0        1        2         3         4         5  \
Cluster ProductIndex                                                            
0       0               306.0    636.0    881.0    1115.0    1396.0    1794.0   
        1                86.0    181.0    288.0     390.0     512.0     655.0   
        2              1629.0   2912.0   4310.0    5838.0    7329.0    9578.0   
        3               102.0    186.0    297.0     372.0     489.0     584.0   
1       0               914.0   1889.0   2247.0    3355.0    3680.0    5329.0   
...                       ...      ...      ...       ...       ...       ...   
98      3              8242.0  14774.0  19826.0   29405.0   37847.0   42040.0   
99      0              7181.0  16022.0  22993.0   29214.0   38109.0   47021.0   
        1              6266.0  12647.0  19348.0   24111.0   31909.0   38150.0   
        2             33689.0  65734.0  97468.0  137696.0  165996.0  207325.0   
        3              8101.0  16464.0  22432.0   31636.0   36900.0   42282.0   

                             6         7         8         9  
Cluster ProductIndex                                          
0       0               1987.0    2277.0    2563.0    2833.0  
        1                677.0     822.0     933.0    1017.0  
        2              10426.0   12454.0   13778.0   15378.0  
        3                678.0     775.0     826.0     951.0  
1       0               5811.0    6412.0    7246.0    8153.0  
...                        ...       ...       ...       ...  
98      3              52269.0   55970.0   64165.0   72093.0  
99      0              55742.0   60633.0   69562.0   76960.0  
        1              44889.0   51153.0   58160.0   63360.0  
        2             235193.0  257282.0  302754.0  330634.0  
        3              53566.0   58031.0   68972.0   74836.0  

[400 rows x 10 columns]

In [ ]:
# v[0,0,0,0]

v_0_0_0_0

In [ ]:
# supplier_df

,SupplierId,SupplierEasting,SupplierNorthing,SupplierProductGroup,SupplierCapacity,SupplierVehicleType
0,1.0,165124.0,823186.0,1.0,1708620.0,0.0
1,2.0,218457.0,712442.0,1.0,1410738.0,0.0
2,3.0,166352.0,777690.0,1.0,434234.0,1.0
3,4.0,370643.0,797391.0,1.0,596810.0,1.0
4,5.0,241260.0,687436.0,1.0,233900.0,1.0
5,6.0,307598.0,607916.0,1.0,460456.0,1.0
6,7.0,365265.0,741339.0,1.0,1583804.0,0.0
7,8.0,393634.0,656828.0,1.0,797146.0,1.0
8,9.0,301473.0,722990.0,1.0,1384516.0,0.0
9,10.0,322640.0,691342.0,1.0,997482.0,0.0


In [ ]:
asdfasdf

NameError: name 'asdfasdf' is not defined

In [ ]:
# # Create the model
# model2 = xp.problem(name = 'MEWLP2')
# # Sets and Notation
# S = list(supplier_df.index) # Supplier Index
# S_P0 = list(supplier_df[supplier_df['SupplierProductGroup'] == 0]['SupplierProductGroup'].index)
# S_P1 = list(supplier_df[supplier_df['SupplierProductGroup'] == 1]['SupplierProductGroup'].index)
# S_P2 = list(supplier_df[supplier_df['SupplierProductGroup'] == 2]['SupplierProductGroup'].index)
# S_P3 = list(supplier_df[supplier_df['SupplierProductGroup'] == 3]['SupplierProductGroup'].index)
# S_P_dict = {0:S_P0, 1:S_P1, 2:S_P2, 3:S_P3}

# W = list(candidate_df.index) # Warehouse Index
# C = list(cluster_center_df.index) # Cluster Index
# P = list(agg_dem_cus_period_df.reset_index()['ProductIndex'].unique())
# T = list(list(agg_dem_cus_period_df.columns))

# # Output Variables
# x = np.array([xp.var(f'x_{w}_{c}_{t}', vartype = xp.binary) for w in W for c in C for t in T], dtype = xp.npvar).reshape(len(W), len(C), len(T))
# y = np.array([xp.var(f'y_{w}_{t}', vartype = xp.binary) for w in W for t in T], dtype = xp.npvar).reshape(len(W), len(T))
# o = np.array([xp.var(f'o_{w}', vartype = xp.binary) for w in W], dtype = xp.npvar).reshape(len(W))
# v = np.array([xp.var(f'v_{w}_{c}_{p}_{t}', vartype = xp.continuous, lb = 0) for w in W for c in C for p in P for t in T], dtype = xp.npvar).reshape(len(W), len(C), len(P), len(T))
# z = np.array([xp.var(f'z_{w}_{s}_{t}', vartype = xp.continuous, lb = 0) for w in W for s in S for t in T], dtype = xp.npvar).reshape(len(W), len(S), len(T))

# model2.addVariable(x, y, o, v, z)

# # Constraints 
# for w in W:
#     Capacity_W = candidate_df.loc[w,'Capacity']
#     for t in T:
#         model2.addConstraint(xp.constraint(o[w] >= y[w,t]))
#         model2.addConstraint(xp.constraint(xp.Sum(v[w,c,p,t] for c in C for p in P) <= Capacity_W * y[w,t]))
#         if t != 0:
#             model2.addConstraint(xp.constraint(y[w, t] >= y[w,t-1]))    

#         for p in P:
#             model2.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for s in S_P_dict[p]) == xp.Sum(v[w,c,p,t] for c in C)))
#         for c in C:
#             model2.addConstraint(xp.constraint(x[w,c,t] <= y[w,t]))  

# for t in T:
#     for c in C:
#         model2.addConstraint(xp.constraint(xp.Sum(x[w,c,t] for w in W) == 1))
#         # model2.addConstraint(xp.constraint(xp.Sum(x[w,c,t] for w in W) >= 1))
#         for p in P:
#             Demand = agg_dem_cus_period_df.loc[(c,p),t]
#             # model2.addConstraint(xp.constraint(xp.Sum(v[w,c,p,t] for w in W) >= Demand))
#             for w in W:
#                 model2.addConstraint(xp.constraint(v[w,c,p,t] == Demand * x[w,c,t]))    
#                 # model2.addConstraint(xp.constraint(v[w,c,p,t] <= Demand * x[w,c,t]))    

# for s in S:
#     Capacity_S = supplier_df.loc[s,'SupplierCapacity']
#     # Product_S = supplier_df.loc[s,'SupplierProductGroup']
#     for t in T:
#         model2.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for w in W) <= Capacity_S))



# Setup_cost = xp.Sum(candidate_df.loc[w,'Setup'] * o[w] for w in W)
# Operating_cost = xp.Sum(candidate_df.loc[w,'Operating'] * y[w,t] for w in W for t in T)
# Tra_w_s_cost = xp.Sum(cost_w_to_s.loc[w,s] * z[w,s,t] for w in W for s in S for t in T)
# Tra_w_c_cost = xp.Sum(cost_w_to_cluster.loc[w,c] * v[w,c,p,t] for w in W for c in C for p in P for t in T)

# obj = Setup_cost + Operating_cost + Tra_w_s_cost + Tra_w_c_cost

# model2.setObjective(obj, sense = xp.minimize)
# model2.setControl('miprelstop', 1e-3)
# model2.setControl('maxtime', 3600)
# tic_time = time.time()
# # Solve the problem
# model2.solve()
# toc_time = time.time()
# solve_time = toc_time - tic_time
# obj_value = model2.getObjVal()

# mip_gap_percent = 100*(obj_value - model2.getAttrib('bestbound'))/obj_value
# print(f'Solving Time: {solve_time}')
# print(f'Objective Value: {obj_value}')
# print(f'%Gaps: {mip_gap_percent}')

FICO Xpress v9.3.5, Hyper, solve started 23:10:01, Feb 12, 2025
Heap usage: 671MB (peak 671MB, 160MB system)
Minimizing MILP MEWLP2 using up to 11 threads and up to 18GB memory, with these control settings:
MAXTIME = 3600
OUTPUTLOG = 1
MIPRELSTOP = .001
Original problem has:
   1131390 rows      1338040 cols      4667520 elements    224840 entities
Presolved problem has:
    242700 rows       453310 cols      2204047 elements    220110 entities
LP relaxation tightened
Presolve finished in 10 seconds
Heap usage: 958MB (peak 1583MB, 160MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  8.00e+06] / [ 1.07e-02,  1.31e+02]
  RHS and bounds [min,max] : [ 1.00e+00,  1.53e+07] / [ 1.00e+00,  1.87e+03]
  Objective      [min,max] : [ 2.45e-04,  6.25e+06] / [ 2.51e-01,  6.88e+06]
Autoscaling applied Curtis-Reid scaling

Will try to keep branch and bound tree memory usage below 14.2GB
Starting concurrent solve with dua

In [ ]:
# x_matrix = model2.getSolution(x)
# y_matrix = model2.getSolution(y)
# v_matrix = model2.getSolution(v)
# z_matrix = model2.getSolution(z)

In [ ]:
# w_open_output = pd.DataFrame(y_matrix).astype(int)


In [ ]:
# w_flow_output = pd.DataFrame()
# for w in W:
#     Capacity_W = candidate_df.loc[w,'Capacity']
#     if np.any((Capacity_W - pd.DataFrame(v_matrix[w,:,:,:].sum(axis = 0)).sum(axis = 0)) < 0) :
#         raise ValueError(f'Warehose {w} exceed its capacity.')
#     else:
#         for c in C:
#             sub_df = pd.DataFrame(v_matrix[w,c,:,:])
#             sub_df.reset_index(inplace= True)
#             sub_df.rename(columns = {'index':'ProductIndex'},inplace = True)
#             sub_df['CandidateIndex'] = w
#             sub_df['ClusterIndex'] = c

#             w_flow_output = pd.concat([w_flow_output, sub_df])

# w_flow_output = w_flow_output.groupby(['CandidateIndex','ClusterIndex','ProductIndex']).sum()

Solving Time: 246.52578496932983
Objective Value: 40049921.54336245
%Gaps: 0.09978364650600091


In [ ]:
# cover_output = agg_dem_cus_period_df.copy()
# cover_output.loc[:,:] = -1
# for c in C:
#     for p in P:
#         for t in T:
#             cover_from_w = np.where(x_matrix[:,c,t] >= 0.0001)[0]
#             cover_from_w_check = np.where(v_matrix[:,c,p,t] >= 0.001)[0]
#             if np.size(cover_from_w) == 1 and cover_from_w == cover_from_w_check: # Check that only one w will c for p during t 
#                 cover_output.at[(c,p),t] = int(cover_from_w[0])
#             else:
#                 raise ValueError(f"More Than 2 warehouse covers customer {c} for product {p} during time {t}")

# cover_output = cover_output.astype('int')  

# if np.any(cover_output == -1):
#     raise ValueError('Some entry of this df has not filled in yet.')

In [ ]:
# s_flow_output = pd.DataFrame()
# for s in S:
#     Capacity_S = supplier_df.loc[s,'SupplierCapacity']
#     if np.any(np.round(Capacity_S - pd.DataFrame(z_matrix[:,s,:].sum(axis = 0)),6) < 0) :
#         raise ValueError(f'Supplier {s} exceed its capacity.')
#     else:
#         for w in W:
#             sub_df = pd.DataFrame(z_matrix[w,s,:])
#             sub_df.reset_index(inplace= True)
#             sub_df.rename(columns = {'index':'ProductIndex'},inplace = True)
#             sub_df['CandidateIndex'] = w
#             sub_df['SupplierIndex'] = s

#             s_flow_output = pd.concat([s_flow_output, sub_df])            

# s_flow_output = s_flow_output.groupby(['SupplierIndex','CandidateIndex','ProductIndex']).sum()

In [ ]:
# s = 1
# Capacity_S = supplier_df.loc[s,'SupplierCapacity']
# Capacity_S - pd.DataFrame(z_matrix[:,s,:].sum(axis = 0))

,0
0,1.410738e+06
1,1.023566e+06
2,8.381920e+05
3,6.501870e+05
4,6.132410e+05
5,6.727820e+05
6,5.299210e+05
7,3.442510e+05
8,0.000000e+00
9,-2.328306e-10


In [ ]:
# np.where(np.round(Capacity_S - pd.DataFrame(z_matrix[:,s,:].sum(axis = 0)),6) < 0)

(array([], dtype=int64), array([], dtype=int64))

In [ ]:
# cover_output

0    1    2    3    4    5    6    7    8    9
Cluster ProductIndex                                                  
0       0              18   18  296  296  296  296  296   18  296   18
        1              18   18  296  296  296  296  296   18  296   18
        2              18   18  296  296  296  296  296   18  296   18
        3              18   18  296  296  296  296  296   18  296   18
1       0             314  182  182  182  182  182  182  338  338  182
...                   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
48      3             314  182  182  182  182   47   47   47   47  314
49      0             314  182  182  182  182  182  182  182  182  296
        1             314  182  182  182  182  182  182  182  182  296
        2             314  182  182  182  182  182  182  182  182  296
        3             314  182  182  182  182  182  182  182  182  296

[200 rows x 10 columns]

In [ ]:
# x_matrix[314,3,9]

0.9999999999990538

In [ ]:
# np.where(x_matrix[:,3,9] != 0)

(array([296, 314]),)

In [ ]:
# np.where(w_flow_output.loc[(0,3,0):(440,3,0),9] != 0)[0]

array([ 3588,  3589,  3590,  3591,  3596,  3597,  3598,  3599,  3608,
        3609,  3610,  3611,  3612,  3613,  3614,  3615,  3616,  3617,
        3618,  3619,  3620,  3621,  3622,  3623,  3624,  3625,  3626,
        3627,  3628,  3629,  3630,  3631,  3636,  3637,  3638,  3639,
        3648,  3649,  3650,  3651,  3656,  3657,  3658,  3659,  3668,
        3669,  3670,  3671,  3672,  3673,  3674,  3675,  3676,  3677,
        3678,  3679,  3680,  3681,  3682,  3683,  3684,  3685,  3686,
        3687,  3688,  3689,  3690,  3691,  3728,  3729,  3730,  3731,
        3732,  3733,  3734,  3735,  3736,  3737,  3738,  3739,  3740,
        3741,  3742,  3743,  3748,  3749,  3750,  3751,  3752,  3753,
        3754,  3755,  3760,  3761,  3762,  3763,  3776,  3777,  3778,
        3779,  9440,  9441,  9442,  9443,  9512,  9513,  9514,  9515,
        9516,  9517,  9518,  9519,  9520,  9521,  9522,  9523,  9564,
        9565,  9566,  9567, 10024, 10025, 10026, 10027, 10120, 10121,
       10122, 10123,

3

In [ ]:
# np.where(v_matrix[:,0,0,0] != 0)[0]

array([18])

In [ ]:
# np.where(x_matrix[:,0,0] != 0)[0]

array([18])

In [ ]:
# np.where(v_matrix[:,c,p,t] != 0)[0]

array([296, 314])

In [ ]:
# len(np.where(w_flop_output != 0)[0])

15956

In [ ]:
# cover_from_w

array([18])

In [ ]:
# cover_from_w_check

array([], dtype=int64)

In [ ]:
c

99

In [ ]:
t

0

In [ ]:
np.where(v_matrix[:,c,p,t] != 0)

(array([], dtype=int64),)

In [ ]:
cover_from_w_check

array([], dtype=int64)

In [ ]:
np.where(x_matrix[:,c,t] != 0)[0]

array([338])

In [ ]:
np.where(v_matrix[:,c,0,t] != 0)[0]

array([338])

In [ ]:
# model2.solve()
# toc_time = time.time()
# solve_time = toc_time - tic_time
# obj_value = model2.getObjVal()

FICO Xpress v9.3.5, Hyper, solve started 16:59:29, Feb 12, 2025
Heap usage: 1258MB (peak 1258MB, 1000MB system)
Minimizing MILP MEWLP2 using up to 11 threads and up to 18GB memory, with these control settings:
MAXTIME = 600
OUTPUTLOG = 1
MIPRELSTOP = .001
Original problem has:
   2231890 rows      2438040 cols      8847520 elements    444840 entities
Presolved problem has:
    464867 rows       674977 cols      3977383 elements    441777 entities
LP relaxation tightened
Presolve finished in 23 seconds
Heap usage: 1834MB (peak 3096MB, 1000MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  8.00e+06] / [ 3.77e-03,  2.63e+02]
  RHS and bounds [min,max] : [ 1.00e+00,  1.53e+07] / [ 1.00e+00,  3.74e+03]
  Objective      [min,max] : [ 2.45e-04,  6.25e+06] / [ 1.25e-01,  6.88e+06]
Autoscaling applied Curtis-Reid scaling

Will try to keep branch and bound tree memory usage below 11.0GB
Starting concurrent solve with